In this notebook I am exporting all of the player Fanduel data for the 2020 NBA season. This information was pulled from this site http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=10&day=22&year=2019 using Beautiful soup and then by iterating over the dropdown menu.

While I want to do additional investigation by cleaning the table up to include stats, this first go around was to prove out that I could pull the records for each night of games and then merge them by date and by player to be able to look at the aggregate information for Fanduel points scored vs fanduel price, with some other pieces of information such as team, opponent, and whether the player was on the road vs at home.

In [1]:
#pulling all relevant libraries 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

the below information was used to scrape the site. I ran this once and then commneted it out after saving the tables as a csv to avoid constantly pulling the data from the web.

In [2]:
##realized the url just updated based on dates, so wrote a loop to accumulate all of the urls
#years = [2019,2020]
#months = list(range(1,13,1))
#days = list(range(1,32,1))

#urls = []
#for year in years:
#    for month in months:
#        for day in days:
#            urls.append(str('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon='+str(month)+'&day='+str(day)+'&year='+str(year)))

##The 2020 season kicked off October 22, 2019 so I want to have a list with just the information for those dates
##finding index number of url that matches that day
#urls.index('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=10&day=22&year=2019')
##ended up being 300
##finding index that matches last game of regular season - March 11. I am not counting playoffs or post pandemic games sense not all teams are playing 
#urls.index('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=10&year=2020')
##ended up being 443
##saving list just for dates that would be in 2020 season
#seasonurls = urls[300:444]

##we should now have the page we need for each url

This is the script to scrape all of the player data. I only ran this once and saved it as a csv in the section in the code marked below because running it is very time consuming.

In [3]:
##creating an empty list that will have the player row for each table

#rows = []

##Running a loop through all the urls to get every piece of data in one list
#scraping each url
#for url in urls:
#    webpage = requests.get(url)
#    webpage_content = webpage.content
#    soup = BeautifulSoup(webpage_content,'html.parser')
#    table_rows = soup.find_all('td')
    #pulling just the player data, which starts on the 24th entry. 
    #We also want to add some kind of date identifier so for now im doing the url
#    for row in table_rows[24:]:
#        rows.append([row.get_text(),url])

In [4]:
##create data frame
#sample_frame = pd.DataFrame.from_records(rows).reset_index()
##save to csv
#sample_frame.to_csv('All_rows.csv')

The above text can be uncommented if you ever need to pull the information again, for example in 2021. The below information is now pulling from the "All_rows" CSV I saved, which has all data from the beginning of the 2021 season through March 10, the day before games were halted due to COVID.<br><br>
From here on, I'm going to approach the problem using that static csv, starting with creating a data frame

In [5]:
#create frame from saved csv
new_frame = pd.read_csv('All_rows.csv')
#rename columns
new_frame.columns = ['Row','ID','Data','URL']

In [6]:
#clean up column with urls so it just has date
dates = new_frame['URL'].map(lambda x: x.replace('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=','')\
                             .replace('&year=','/')\
                             .replace('&day=','/'))
new_frame['Date'] = dates

#create separate frame that removes all the url columns
date_frame = new_frame[['Data','Date']].reset_index()
len(date_frame)

580661

Everything from here to the next markup is me clearing out rows that were in the table but did not have player information.

In [7]:
#We need to remove some ads and nav stuff here by 
#converting to a series, finding the ones that match, and adding back to the table
find_Ads = date_frame['Data']
#create series that has 0 for what matches the ads
ads_found = find_Ads.str.find('RotoGuru')
#add column to table with 0's
date_frame['Ad'] = ads_found
#create new table with those rows with zero gone
no_ads = date_frame[date_frame['Ad'] != 1].reset_index()
len(no_ads)

580320

In [8]:
#removing index columns
del no_ads['index']
del no_ads['level_0']

In [9]:
#Repeating to remove Jump To:
#converting to a series, finding the ones that match, and adding back to the table
find_jump = no_ads['Data']
#create series that has 0 for what matches the text
jump_found = find_jump.str.find('Jump to:')
#add column to table with 0's
no_ads['Remove'] = jump_found
#create new table with those rows with zero gone
jump_gone = no_ads[no_ads['Remove'] !=0].reset_index()
len(jump_gone)

579475

In [10]:
#There's a term 'Unlisted' that pops up occasionally and breaks everything. I'm clearing that here
find_unlisted = jump_gone['Data']
#create series that has a 0 for where it says unlisted
unlisted_found = find_unlisted.str.find('Unlisted')
#add column to table with 0's
jump_gone['Z'] = unlisted_found
#create new table with those rows removed
unlisted_gone = jump_gone[jump_gone['Z'] != 0].reset_index()
len(unlisted_gone)

579238

In [11]:
##There's a subtable headers that aren't player data. we are getting rid of most those here. Min can't go 
#because it lines up with al-farouq aminu
##creates list of all the words I want to find and get rid of
sub = ['Forward','Center','FD Points','Salary','Team','Opp.','Score','Stats','Unlisted']
pattern = '|'.join(sub)

unlisted_gone['gone'] = unlisted_gone['Data'].str.contains(pattern, case=False)


In [12]:
#clearing columns
del unlisted_gone['index']
del unlisted_gone['Remove']
del unlisted_gone['level_0']

In [13]:
#remove any rows where we found those subtable headers
cleaner_table = unlisted_gone[unlisted_gone['gone'] != True].reset_index()
#specifically remove Min
#converting to a series, finding the ones that match, and adding back to the table
find_min = cleaner_table['Data']
#create series that has 0 for what matches the word Min
min_found = find_min.str.find('Min')
#add column to table with 0's
cleaner_table['Remove'] = min_found
#create new table with those rows with zero gone
clean_table = cleaner_table[cleaner_table['Remove'] !=0].reset_index()

In [14]:
#removing extra rows
del clean_table['index']
del clean_table['Ad']
del clean_table['gone']
del clean_table['Remove']



Now that we've cleared all the random stuff out, we're going to try to create a row per player per game and include the date played

In [15]:
##create series with the data
just_data = clean_table[['Data','Date']].reset_index()

In [16]:
del just_data['index']

In [17]:
 #clearing out a row with an "unlisted" entry
just_data.drop([4158])

Data       Date
0                             SG   1/1/2019
1                 Beasley, Malik   1/1/2019
2                           40.7   1/1/2019
3                         $4,500   1/1/2019
4                            den   1/1/2019
...                          ...        ...
572710                       den  9/22/2020
572711                     v lal  9/22/2020
572712                   114-106  9/22/2020
572713                      4:35  9/22/2020
572714     0pt 2rb 1as 1to 0-1fg  9/22/2020

[572714 rows x 2 columns]

Now we have a frame with every piece of information and a date, but we need to make it one row per player per game

In [18]:
#merging data and date in a column so
#I can then hopefully turn each one into a series and then just have the date once at the end.
just_data['merge_date'] = just_data['Data'].astype(str)+'|'+just_data['Date']

In [19]:
#turning my merged column into a list so I can run a comprehension and then add the date to the end of a player row
just_datas = just_data['merge_date']
just_datas[4158]


'SG|1/3/2019'

In [20]:
data_list = list(just_datas)

In [21]:
#This is a thing that let you make a row per player, but you couldn't add the date
##each row was 9 entries. So we're gonna write this thing to create a series with lists of 9 entries each. This gets thrown off very easily though
players = [data_list[x:x+9] for x in range(0, len(data_list), 9)]
players

[['SG|1/1/2019',
  'Beasley, Malik|1/1/2019',
  '40.7|1/1/2019',
  '$4,500|1/1/2019',
  'den|1/1/2019',
  'v nyk|1/1/2019',
  '\xa0115-108|1/1/2019',
  '29:56|1/1/2019',
  ' \xa0 23pt 6rb 5as 1st 5trey 8-15fg 2-2ft|1/1/2019'],
 ['SG|1/1/2019',
  'Hield, Buddy^|1/1/2019',
  '40|1/1/2019',
  '$6,900|1/1/2019',
  'sac|1/1/2019',
  'v por|1/1/2019',
  '\xa0108-113|1/1/2019',
  '39:02|1/1/2019',
  ' \xa0 27pt 5rb 4as 1bl 2to 5trey 11-23fg|1/1/2019'],
 ['PG|1/1/2019',
  'Lillard, Damian^|1/1/2019',
  '39.2|1/1/2019',
  '$9,200|1/1/2019',
  'por|1/1/2019',
  '@ sac|1/1/2019',
  '\xa0113-108|1/1/2019',
  '41:54|1/1/2019',
  ' \xa0 25pt 6rb 6as 1st 5to 1trey 8-21fg 8-8ft|1/1/2019'],
 ['PG|1/1/2019',
  'Simmons, Ben^|1/1/2019',
  '38.8|1/1/2019',
  '$9,800|1/1/2019',
  'phi|1/1/2019',
  '@ lac|1/1/2019',
  '\xa0119-113|1/1/2019',
  '32:50|1/1/2019',
  ' \xa0 14pt 9rb 8as 1st 1bl 4to 6-13fg 2-6ft|1/1/2019'],
 ['SG|1/1/2019',
  'Bogdanovic, Bogdan|1/1/2019',
  '37.5|1/1/2019',
  '$6,100|1/1/2019',

In [22]:
#now we have a row per player, but every piece of data has the date. So we want to just make that its own entry at the end
for player in players:
    player.insert(0,player[0].split('|')[1])


In [23]:
players

[['1/1/2019',
  'SG|1/1/2019',
  'Beasley, Malik|1/1/2019',
  '40.7|1/1/2019',
  '$4,500|1/1/2019',
  'den|1/1/2019',
  'v nyk|1/1/2019',
  '\xa0115-108|1/1/2019',
  '29:56|1/1/2019',
  ' \xa0 23pt 6rb 5as 1st 5trey 8-15fg 2-2ft|1/1/2019'],
 ['1/1/2019',
  'SG|1/1/2019',
  'Hield, Buddy^|1/1/2019',
  '40|1/1/2019',
  '$6,900|1/1/2019',
  'sac|1/1/2019',
  'v por|1/1/2019',
  '\xa0108-113|1/1/2019',
  '39:02|1/1/2019',
  ' \xa0 27pt 5rb 4as 1bl 2to 5trey 11-23fg|1/1/2019'],
 ['1/1/2019',
  'PG|1/1/2019',
  'Lillard, Damian^|1/1/2019',
  '39.2|1/1/2019',
  '$9,200|1/1/2019',
  'por|1/1/2019',
  '@ sac|1/1/2019',
  '\xa0113-108|1/1/2019',
  '41:54|1/1/2019',
  ' \xa0 25pt 6rb 6as 1st 5to 1trey 8-21fg 8-8ft|1/1/2019'],
 ['1/1/2019',
  'PG|1/1/2019',
  'Simmons, Ben^|1/1/2019',
  '38.8|1/1/2019',
  '$9,800|1/1/2019',
  'phi|1/1/2019',
  '@ lac|1/1/2019',
  '\xa0119-113|1/1/2019',
  '32:50|1/1/2019',
  ' \xa0 14pt 9rb 8as 1st 1bl 4to 6-13fg 2-6ft|1/1/2019'],
 ['1/1/2019',
  'SG|1/1/2019',
  

In [24]:
#creating list that has each player entry as its own record without date. 
#Note - it's throwing an index error, but I'm still getting all the information I'll need. I cannot figure out why
#the error is occurring especially considering it is still able to do what I'm asking
player_rows =[]
for player in players:
    if len(player) == 10:
        player_rows.append([player[0],\
                            player[1].split('|')[0],\
                            player[2].split('|')[0],\
                            player[3].split('|')[0],\
                            player[4].split('|')[0],\
                            player[5].split('|')[0],\
                            player[6].split('|')[0],\
                            player[7].split('|')[0],\
                            player[8].split('|')[0],\
                            player[9].split('|')[0]])
                            
        



In [25]:
sample_frame = pd.DataFrame.from_records(player_rows).reset_index()


We now have a frame with records for every player for every game. It's time to clean things up!

In [26]:
sample_frame.columns = ['Row','Date,','Position','Name','Fanduel_Points','Fanduel_Price','Team','Opponent','Score','Minutes','Stats']
sample_frame.head(1)

Row     Date, Position            Name Fanduel_Points Fanduel_Price Team  \
0    0  1/1/2019       SG  Beasley, Malik           40.7        $4,500  den   

  Opponent     Score Minutes                                   Stats  
0    v nyk   115-108   29:56     23pt 6rb 5as 1st 5trey 8-15fg 2-2ft

I'm now going to clean up the price column, and then use that column to delete non player rows by making a new frame that drops any rows that do not have a price

In [27]:
#Replace currency symbols in column so we can make it an integer
sample_frame['Fanduel_Price'] = sample_frame['Fanduel_Price'].str.replace(',', '')
sample_frame['Fanduel_Price'] = sample_frame['Fanduel_Price'].str.replace('$', '')
#Turn the column to integers
sample_frame['Fanduel_Price'] = pd.to_numeric(sample_frame['Fanduel_Price'], errors='coerce')

*You are here. You're trying to do the minutes piece but you're getting an index error.

In [28]:
#there is no price listed for some players. i am removing them since the whole goal is to see who exceeds their price
df1 = sample_frame[~sample_frame['Fanduel_Price'].isna()].reset_index()

In [29]:
#convert DNP in Minutes column to 0's. Had to make 0:0 to allow split function to work for players with actual minutes
df1['NoDNP'] = df1['Minutes'].apply(lambda x: '0:0' if str(x) == 'DNP' else x)
#convert NA in Minutes column to 0's
df1['NoNaN'] = df1['NoDNP'].apply(lambda x: '0:0' if str(x) == 'nan' else x)
#convert mm:ss to total minutes as a float by splitting and then adding
df1['Minutes_Played']= df1['NoNaN'].str.split(':').apply(lambda x: int(x[0]) + ((int(x[1])/60)))

In [30]:
#Convert Fanduel_Points to float
df1['Fanduel_Points'] = pd.to_numeric(df1['Fanduel_Points'])

In [31]:
#create column for home vs away and updated column for opponent
df1['Split'] = df1['Opponent'].str[0]
df1['Home'] = df1.Split.apply(lambda x: 'Home' if str(x) == 'v' else 'Away')
df1['Foe'] = df1['Opponent'].str[1:]

In [32]:
#clearing out some columns
del df1['index']
del df1['Row']
del df1['NoDNP']
del df1['NoNaN']
del df1['Split']
del df1['Opponent']

In [33]:
#get rid of some carrots that are appearing
df1['Name'] = df1['Name'].str.replace('^','')

One day, I'll fix it up so the stats are their own columns, but I'd like to get into finding out some value per minute and things like that so we're gonna make a tight table here and start poking around.

In [34]:
tight_frame = df1[['Date,','Position','Name','Fanduel_Points','Fanduel_Price','Team','Minutes_Played','Home','Foe']].reset_index()
del tight_frame['index']
tight_frame.columns = ['Date','Position','Name','Fanduel_Points','Fanduel_Price','Team','Minutes_Played','Home_or_Away','Opponent']
#going ahead and saving this for a safe template
tight_frame.to_csv('tight_frame.csv')